In [2]:
import json
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
data = []
with open("drone_sense_cse_base_1.txt", "r") as file:
    for line in file:
        data.append(json.loads(line)["answer"])

df = pd.DataFrame(data)
df.head()

,datenow,timenow,rangeIdx,dopplerIdx,numDetectedObj,x_coord,y_coord,z_coord,rp_y,noiserp_y,interFrameProcessingTime,interFrameProcessingMargin,interChirpProcessingMargin,transmitOutputTime,activeFrameCPULoad,interFrameCPULoad,snrDB,noiseDB,activity
0,27/9/2024,18_28_49,"[16, 16]","[-1, -1]",2,"[-0.479915589094162, 0.06544303148984909]","[0.48412030935287476, 0.6793357729911804]","[-0.15031613409519196, -0.1466507613658905]","[42.61455876118234, 41.924698354452374, 41.297...","[43.61799208006228, 41.32890982136742, 38.2872...",6172,78325,0,13360,0,6,"[14.9, 14.9]","[45.400000000000006, 45.400000000000006]",drone_sense_cse_base_1
1,27/9/2024,18_28_49,NaN,NaN,0,[],[],[],"[40.20004733762748, 40.48226295856247, 40.7958...","[40.70176399706746, 40.48226295856247, 41.7051...",6091,78243,0,13718,0,6,NaN,NaN,drone_sense_cse_base_1
2,27/9/2024,18_28_49,NaN,NaN,0,[],[],[],"[40.952622326787434, 42.269628557817356, 38.69...","[39.917831716692504, 41.10940878286243, 35.465...",6091,78325,0,13360,0,6,NaN,NaN,drone_sense_cse_base_1
3,27/9/2024,18_28_49,NaN,NaN,0,[],[],[],"[42.61455876118234, 39.00847027145757, 41.2661...","[39.886474425477516, 38.977112980242566, 39.22...",6091,78325,0,13360,0,6,NaN,NaN,drone_sense_cse_base_1
4,27/9/2024,18_28_49,NaN,NaN,0,[],[],[],"[42.144199392957375, 39.19661401874755, 38.412...","[41.45433898622741, 39.353400474822536, 36.249...",6091,78325,0,13360,0,6,NaN,NaN,drone_sense_cse_base_1


In [5]:
print(df.shape)
df = df.dropna()
print(df.shape)
print(df["x_coord"])

(1717, 19)
(1544, 19)
0               [-0.479915589094162, 0.06544303148984909]
6              [0.13088606297969818, -0.5017299056053162]
7                                    [-0.479915589094162]
12                                   [-0.479915589094162]
19                                   [-0.479915589094162]
                              ...                        
1712    [-4.208805084228516, -3.7956957817077637, -3.3...
1713    [-4.208805084228516, -3.2966926097869873, -3.7...
1714    [-0.4581012427806854, -5.5735650062561035, -3....
1715    [-5.5735650062561035, -2.429572582244873, -3.6...
1716            [-4.035653591156006, -2.4786548614501953]
Name: x_coord, Length: 1544, dtype: object


In [13]:
# point_cloud = np.column_stack((df["x_coord"], df["y_coord"], df["z_coord"]))

# print(point_cloud.shape)  # Should output: (100, 3)
# print(point_cloud[0])
# df["x_coord"].shape

(199, 3)
[list([0.09543775767087936, -0.13361285626888275, 0.16224418580532074, 0.09543775767087936, -0.13361285626888275, -0.46355482935905457, 0.08180379122495651])
 list([0.5925818085670471, 0.34047821164131165, 0.7010554671287537, 0.6014037132263184, 0.1581762731075287, 0.7118168473243713, 0.802276074886322])
 list([0.11321236938238144, -0.48918384313583374, 0.17971837520599365, 0.04778941720724106, -0.5746359825134277, -0.199546217918396, 0.3332360088825226])]


(199,)

In [11]:
x_coords = [e for elm in df['x_coord'][0:3] for e in elm]
x_coords

[-0.479915589094162,
 0.06544303148984909,
 0.13088606297969818,
 -0.5017299056053162,
 -0.479915589094162]

In [ ]:
for index,row in df.iterrows():
    # frameIDX = np.random.randint(0, pointcloudRadarDepth.shape[0])
    # frameIDX = index
    # distancesRadar = np.linalg.norm(pointcloudRadarDepth["radarPCD"][frameIDX], axis=1)
    # normalized_distances = (distancesRadar - distancesRadar.min()) / (distancesRadar.max() - distancesRadar.min())
    # c = np.linspace(0, 1,7)
    sns.set(style="whitegrid")
    fig = plt.figure(figsize=(12,6))
    ax1 = fig.add_subplot(111,projection='3d')
    # distancesRadar = np.linalg.norm(pointcloudRadarDepth["radarPCD"][frameIDX], axis=1)
    # normalized_distancesRadar = (distancesRadar - distancesRadar.min()) / (distancesRadar.max() - distancesRadar.min())
    frames=10
    img1 = ax1.scatter([e for elm in df['x_coord'][index:index+frames] for e in elm], [e for elm in df['y_coord'][index:index+frames] for e in elm], [e for elm in df['z_coord'][index:index+frames] for e in elm], cmap = [e for elm in df['dopplerIdx'][index:index+frames] for e in elm], marker='o')
    fig.colorbar(img1,ax=ax1, label='Intensity')
    ax1.set_title('Radar PCD')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_xlim(-10,10)
    ax1.set_ylim(0,10)
    ax1.set_zlim(-3,10)
    plt.tight_layout()
    plt.savefig(f"./visualization/test/test_{str(index)}.png")
    # plt.show()
    plt.close()
    # break
    

C:\Users\halde\AppData\Local\Temp\ipykernel_17192\3188933831.py:13: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  img1 = ax1.scatter([e for elm in df['x_coord'][index:index+frames] for e in elm], [e for elm in df['y_coord'][index:index+frames] for e in elm], [e for elm in df['z_coord'][index:index+frames] for e in elm], cmap = [e for elm in df['dopplerIdx'][index:index+frames] for e in elm], marker='o')
